# Trying Unsupervised binning
Metabat2 version 2.15 bioconda <br>
concoct v1.1.0 bioconda <br>
MaxBin 2.2.7 bioconda <br>
binsanity <br>
dastool 1.1.3 <br>
<br>
all the attempts etc was my trial and error to find the best binning parameter for everything. Once found the best one, binninmethod01 uses it.<br>

MetaBCC-LR version 1.0.0 <br> <br>
We want high completeness, low contamination https://github.com/Ecogenomics/CheckM/issues/65 <br>
MAGs:
- contamination < 10% <br>
- completeness-5\*contamination > 50 <br>
The SH index is worth considering, but isn't nearly as critical as the estimated percentage of contamination. If the SH index is high (ideally 100%), it suggest the majority of contamination is from very similar species and thus any contamination is likely from the pangenome of the species being considered. Alternatively, if the SH index is very low (ideally 0%) this indicates all the contamination is likely from other species (perhaps very divergent species) and thus you may be able to identify it and remove it from the genome.

then https://merenlab.org/2016/06/22/anvio-tutorial-v2/#anvi-import-collection


### Metabat2
https://bitbucket.org/berkeleylab/metabat/src/master/

In [ ]:
conda activate binning

mkdir 05_binning
mkdir 05_binning/metabat2
mkdir 07_checkm

for bam_sorted in $(find ./04_mapped_samples/ -maxdepth 1 -name "*_sorted.bam" -type f); do
    file_name=$(basename $bam_sorted)
    sample_name=${file_name%_*}
    
    # get contig depths
    echo $sample_name
    jgi_summarize_bam_contig_depths --outputDepth 05_binning/metabat2/${sample_name}_depth.txt $bam_sorted
done


for depths in $(find ./05_binning/metabat2/ -name "*_depth.txt" -type f); do
    file_name=$(basename $depths)
    sample_name=${file_name%_*}
    sample_name=${sample_name%_*}
    
    #make folder
    attempt="metabat_$sample_name"
    out_dir="05_binning/metabat2/${attempt}"
    mkdir $out_dir

    #output fasta
    metabat2 -i 03_correction_medaka/${sample_name}/consensus.fasta \
        -a $depths \
        -o $out_dir/bin \
        -t 32 --seed 1 --saveCls --verbose \
        --maxP 97 --minS 75

    #output list
    metabat2 -i 03_correction_medaka/${sample_name}/consensus.fasta \
        -a $depths \
        -o $out_dir/bin_fa_list \
        -t 32 --seed 1 -l \
        --maxP 97 --minS 75

    #do checkm
    checkm lineage_wf -x fa -t 32 --pplacer_threads 32 $out_dir 07_checkm/${attempt}
    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt
done

for depths in $(find ./05_binning/metabat2/ -name "*_depth.txt" -type f); do
    file_name=$(basename $depths)
    sample_name=${file_name%_*}
    sample_name=${sample_name%_*}
    attempt="metabat_$sample_name"

    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt
done


In [ ]:
#metabat02 to dastool tab separated file
R

library("dplyr")

path_list <- dir("./05_binning/metabat2", pattern = "metabat", full.names = T)
path_save <- "./05_binning/"

for (path in path_list){

  bins_list <- list.files(path = path, pattern = "bin_fa_list", full.names = T)

  bins_contigs <- data.frame(contigs = character(), bins = character())

  for (i in 1:length(bins_list)){
    bins_file_temp <- bins_list[i]
    bin_no_temp <- strsplit(bins_file_temp, split = "\\.")[[1]][3]
    bin_name_temp <- paste0("bin_",bin_no_temp)
    bins_list_temp <- read.table(bins_file_temp, stringsAsFactors = F)
    bins_contigs_temp <- data.frame(contigs = bins_list_temp$V1, bins = bin_name_temp, stringsAsFactors = F)
  
    #append
    bins_contigs <- bins_contigs %>% 
     rbind(bins_contigs_temp)
  }

  bins_contigs <- bins_contigs %>% 
    arrange(contigs)

  write.table(bins_contigs, file = paste0(path_save, basename(path), ".txt"), sep = "\t", col.names = F, quote = F, row.names = F)
}


### Concoct
https://concoct.readthedocs.io/en/latest/

In [ ]:

conda activate binning
mkdir 05_binning/concoct

for corr_folder in $(find ./03_correction_medaka/ -name "sample*" -type d); do
    sample_name=$(basename $corr_folder)
    out_dir="./05_binning/concoct"
    out_cut_fa="./05_binning/concoct/contigs_${sample_name}_10K"
    
    # get cut up fasta
    echo $sample_name
    cut_up_fasta.py $corr_folder/consensus.fasta -c 10000 -o 0 --merge_last -b $out_cut_fa.bed > $out_cut_fa.fa
    concoct_coverage_table.py $out_cut_fa.bed ./04_mapped_samples/${sample_name}_sorted.bam > $out_dir/coverage_table_${sample_name}.tsv 
    
    # setup output directory for concoct results
    attempt="concoct_$sample_name"
    out_dir_samp="05_binning/concoct/$attempt"
    mkdir $out_dir_samp

    # run concoct
    concoct --composition_file $out_cut_fa.fa \
        --coverage_file $out_dir/coverage_table_${sample_name}.tsv \
        -b $out_dir_samp -t 16 -s 1 \
        --total_percentage_pca 99 -c 800

    #merge subcontig clustering
    merge_cutup_clustering.py $out_dir_samp/clustering_gt1000.csv > $out_dir_samp/clustering_merged.csv 

    #export fasta
    mkdir $out_dir_samp/fasta_bins
    extract_fasta_bins.py $corr_folder/consensus.fasta \
        $out_dir_samp/clustering_merged.csv \
        --output_path $out_dir_samp/fasta_bins

    #Delete all bins sized less than 500kb
    cp -R $out_dir_samp/fasta_bins $out_dir_samp/fasta_bins_FINAL
    find $out_dir_samp/fasta_bins_FINAL -name "*.fa" -type f -size -500k -delete
    #now only have 244 bins phew

    #checkm
    checkm lineage_wf -x fa -t 16 --pplacer_threads 16 \
        $out_dir_samp/fasta_bins_FINAL \
        07_checkm/${attempt}
    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt
done

for corr_folder in $(find ./03_correction_medaka/ -name "sample*" -type d); do
    sample_name=$(basename $corr_folder)
    # setup output directory for concoct results
    attempt="concoct_$sample_name"
    out_dir_samp="05_binning/concoct/$attempt"
    mkdir $out_dir_samp


    #Delete all bins sized less than 500kb
    cp -R $out_dir_samp/fasta_bins $out_dir_samp/fasta_bins_FINAL
    find $out_dir_samp/fasta_bins_FINAL -name "*.fa" -type f -size -500k -delete
    #now only have 244 bins phew

    #checkm
    checkm lineage_wf -x fa -t 16 --pplacer_threads 16 \
        $out_dir_samp/fasta_bins_FINAL \
        07_checkm/${attempt}
    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt
done


concoct01 = --total_percentage_pca 99 still contaminated liao. But best so far<br>
concoct02 = --total_percentage_pca 99 -c 800<br>
concoct05 = --total_percentage_pca 99 -i 750 same as 02 <br>
concoct06 = --total_percentage_pca 99 -c 1000<br>
concoct07 = --total_percentage_pca 99 -c 5000 not useful, stick with 02<br> 

In [ ]:
R

library("dplyr")

path_list <- dir("./05_binning/concoct", pattern = "concoct_sample", full.names = T)
path_save <- "./05_binning/"

for (path in path_list){
  bins_list <- list.files(path = path, pattern = "bin_fa_list", full.names = T)
  bins_contigs <- data.frame(contigs = character(), bins = character())

  bins_contigs <- read.csv(paste0(path,"/clustering_merged.csv"), stringsAsFactors = F)
  bins <- paste0("bin_",bins_contigs$cluster_id)
  bins_contigs <- bins_contigs %>% 
    mutate(bins = bins) %>% 
    select(-cluster_id) %>% 
    arrange(contig_id)

  write.table(bins_contigs, file = paste0(path_save, basename(path), ".txt"), sep = "\t", col.names = F, quote = F, row.names = F)
}



### Maxbin
see the txt file that you downloaded

In [ ]:
mkdir 05_binning/maxbin

# Get depth txt

R

library("dplyr")
depth_files <- list.files("./05_binning/metabat2/", pattern = "depth", full.names = TRUE)

for (i in depth_files){
  depth <- read.delim(i, stringsAsFactors = F)
  depth <- depth %>% 
    select(-contains(".var")) %>% 
    select(-contains("Depth")) %>% 
    select(-contains("Len"))

  for (i in 2:ncol(depth)){
    depth_temp <- depth[,c(1, i)]
    depth_name <- colnames(depth_temp)[2]
    depth_name <- strsplit(depth_name, "_")[[1]]
    depth_name <- depth_name[1]
    depth_name <- paste0(depth_name,"_depth.txt")
    write.table(depth_temp, file = paste0("05_binning/maxbin/", depth_name), sep = "\t", col.names = F, quote = F, row.names = F)
  }
}

# MaxBin
for corr_folder in $(find ./03_correction_medaka/ -name "sample*" -type d); do
    sample_name=$(basename $corr_folder)
    attempt="maxbin_${sample_name}"
    mkdir 05_binning/maxbin/${attempt}

    run_MaxBin.pl \
        -contig $corr_folder/consensus.fasta \
        -out 05_binning/maxbin/${attempt}/bin \
        -thread 16 \
        -abund 05_binning/maxbin/${sample_name}_depth.txt \
        -prob_threshold 0.99

    checkm lineage_wf -x fasta -t 16 --pplacer_threads 16 \
        05_binning/maxbin/${attempt}/ \
        07_checkm/${attempt}
    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt

    # Get bins list
    outdir="./05_binning/maxbin/${attempt}/bin_deets"
    mkdir $outdir

    for bins in $(find ./05_binning/maxbin/${attempt}/ -maxdepth 1 -name "*.fasta" -type f); do
        bin_name=$(basename $bins)
        bin_txt=$outdir/${bin_name}.txt
        grep "^>" $bins > $bin_txt
    done
done


maxbin01: -prob_threshold 0.98 <br>
maxbin02: -prob_threshold 0.98 -markerset 40 PAKE INI AJA DEH<br>
maxbin03: -prob_threshold 0.99 <br>
maxbin04: -prob_threshold 0.99 -markerset 40 <br>

In [ ]:
#THE BELOW IN R
R

library("dplyr")

path_list <- dir("./05_binning/maxbin", pattern = "maxbin_sample", full.names = T)
path_save <- "./05_binning/"

for (path in path_list){
  bins_list <- list.files(path = paste0(path,"/bin_deets"), pattern = ".fasta.txt", full.names = T)
  bins_contigs <- data.frame(contigs = character(), bins = character())  


  for (i in 1:length(bins_list)){
  bins_file_temp <- bins_list[i]
  bin_no_temp <- as.numeric(strsplit(bins_file_temp, split = "\\.")[[1]][3])
  bin_name_temp <- paste0("bin_",bin_no_temp)
  bins_list_temp <- read.table(bins_file_temp, stringsAsFactors = F)
  bins_list_temp <- sub('.', '', bins_list_temp$V1)
  
  bins_contigs_temp <- data.frame(contigs = bins_list_temp, bins = bin_name_temp, stringsAsFactors = F)
  
  #append
  bins_contigs <- bins_contigs %>% 
    rbind(bins_contigs_temp)
}

  bins_contigs <- bins_contigs %>% 
    arrange(contigs)

  write.table(bins_contigs, file = paste0(path_save, basename(path), ".txt"), sep = "\t", col.names = F, quote = F, row.names = F)
}





### Binsanity
https://github.com/edgraham/BinSanity/wiki/Usage

In [ ]:
# Binsanity
mkdir binsanity_temp
temp="./binsanity_temp"

for corr_folder in $(find ./03_correction_medaka/ -name "sample*" -type d); do
    sample_name=$(basename $corr_folder)
    out_dir="./05_binning/binsanity"
    attempt="binsanity_${sample_name}"
    
    cp 04_mapped_samples/${sample_name}_sorted* ${temp}/

    Binsanity-profile -i $corr_folder/consensus.fasta \
    -s $temp \
    -o $out_dir/ \
    -c $out_dir/${sample_name} \
    -T 32

    mkdir 05_binning/binsanity/${attempt}
    Binsanity-wf -f $corr_folder -l consensus.fasta \
        -c $out_dir/${sample_name}.cov.x100.lognorm \
        -o $out_dir/${attempt}/ \
        --threads 32 \
        -p -3 --refine-preference -1


    #Got 556 bins wow. Delete all bins sized less than 500kb
    cp -R $out_dir/${attempt}/BinSanity-Final-bins $out_dir/${attempt}/BinSanity-Final-bins-500k
    find $out_dir/${attempt}/BinSanity-Final-bins-500k -name "*.fna" -type f -size -500k -delete
    #now only have 244 bins phew

    #CheckM
    checkm lineage_wf -x fna -t 16 --pplacer_threads 16 \
        $out_dir/${attempt}/BinSanity-Final-bins-500k \
        07_checkm/${attempt}/
    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt

    #Prepare for R
    mkdir $out_dir/${attempt}/BinSanity-Final-bins-500k/bin_deets
    for bins in $(find ./$out_dir/${attempt}/BinSanity-Final-bins-500k/ -maxdepth 1 -name "*.fna" -type f); do
        bin_name=$(basename $bins)
        bin_txt=$out_dir/${attempt}/BinSanity-Final-bins-500k/bin_deets/${bin_name}.txt
        grep "^>" $bins > $bin_txt
    done
    rm $temp/*
done





In [ ]:
#buat indrik ganti
mkdir binsanity_temp
temp="./binsanity_temp"

for corr_folder in $(find ./03_correction_medaka/ -name "sample*" -type d); do
    sample_name=$(basename $corr_folder)
    out_dir="./05_binning/binsanity"
    attempt="binsanity_${sample_name}"
    
    cp 04_mapped_samples/${sample_name}_sorted* ${temp}/

    Binsanity-profile -i $corr_folder/consensus.fasta \
    -s $temp \
    -o $out_dir/ \
    -c $out_dir/${sample_name} \
    -T 32

    mkdir 05_binning/binsanity/${attempt}
    Binsanity-wf -f $corr_folder -l consensus.fasta \
        -c $out_dir/${sample_name}.cov.x100.lognorm \
        -o $out_dir/${attempt}/ \
        --threads 32 \
        -p -3 --refine-preference -1




    #Got 556 bins wow. Delete all bins sized less than 500kb
    cp -R $out_dir/${attempt}/BinSanity-Final-bins $out_dir/${attempt}/BinSanity-Final-bins-500k
    find $out_dir/${attempt}/BinSanity-Final-bins-500k -name "*.fna" -type f -size -500k -delete
    #now only have 244 bins phew

    #CheckM
    checkm lineage_wf -x fna -t 16 --pplacer_threads 16 \
        $out_dir/${attempt}/BinSanity-Final-bins-500k \
        07_checkm/${attempt}/
    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt

    #Prepare for R
    mkdir $out_dir/${attempt}/BinSanity-Final-bins-500k/bin_deets
    for bins in $(find ./$out_dir/${attempt}/BinSanity-Final-bins-500k/ -maxdepth 1 -name "*.fna" -type f); do
        bin_name=$(basename $bins)
        bin_txt=$out_dir/${attempt}/BinSanity-Final-bins-500k/bin_deets/${bin_name}.txt
        grep "^>" $bins > $bin_txt
    done
    rm $temp/*
done

# works normally
Binsanity-wf -f ./ -l sample04_contigs.fasta \
        -c sample04.cov.x100.lognorm \
        -o ./test/ \
        --threads 32 \
        -p -3 --refine-preference -1

# doesn't work
Binsanity-wf -f ./ -l sample03_contigs.fasta \
        -c sample03.cov.x100.lognorm \
        -o ./test/ \
        --threads 32 \
        -p -3 --refine-preference -1

binsanity01 = default. SO HIGH CONTAMINATION <br>
binsanity02 = -p -3 --refine-preference -10 <br>
binsanity03 = -p -3 --refine-preference -3 <br>
binsanity04 = -p -3 --refine-preference -1 THIS IS THE BEST<br>
binsanity05 = -p -1 --refine-preference -1 <br>


In [ ]:
#THE BELOW IN R
R

library("dplyr")

path_list <- dir("./05_binning/binsanity", pattern = "binsanity_sample", full.names = T)
path_save <- "./05_binning/"

for (path in path_list){
  bins_list <- list.files(path = paste0(path, "/BinSanity-Final-bins-500k/bin_deets/"), pattern = ".txt", full.names = T)
  bins_contigs <- data.frame(contigs = character(), bins = character())

  for (i in 1:length(bins_list)){
    bins_file_temp <- bins_list[i]
    bin_name_temp <- paste0("bin_",i)
    bins_list_temp <- read.table(bins_file_temp, stringsAsFactors = F)
    bins_list_temp <- sub('.', '', bins_list_temp$V1)
  
    bins_contigs_temp <- data.frame(contigs = bins_list_temp, bins = bin_name_temp, stringsAsFactors = F)
  
    #append
    bins_contigs <- bins_contigs %>% 
      rbind(bins_contigs_temp)
  }

  bins_contigs <- bins_contigs %>% 
    arrange(contigs)

  write.table(bins_contigs, file = paste0(path_save, basename(path), ".txt"), sep = "\t", col.names = F, quote = F, row.names = F)
}




### DASTOOL
Fasta_to_Scaffolds2Bin.sh -e fasta > my_scaffolds2bin.tsv WAHLAO EH BUT THIS STILL DOESN'T WORK HAHA

metabat02, maxbin02, binsanity04, concoct02

In [ ]:
mkdir 05_binning/dastool/

for corr_folder in $(find ./03_correction_medaka/ -name "sample*" -type d); do
    sample_name=$(basename $corr_folder)
    attempt="dastool_$sample_name"
    out_dir="./05_binning/dastool/$attempt"
    mkdir $out_dir

    temp=$(find ./05_binning/ -name "*${sample_name}.txt" -type f | tr '\n' '\,')

    echo "doing DAS_Tool"
    echo $sample_name
    echo $temp
    echo $out_dir/$attempt

    DAS_Tool \
        --bins $temp \
        --contigs $corr_folder/consensus.fasta \
        --outputbasename $out_dir/$attempt \
        --threads 32 --write_bins 1 \
        --search_engine diamond

    checkm lineage_wf -x fa -t 32 --pplacer_threads 32 \
        $out_dir/${attempt}_DASTool_bins \
        07_checkm/${attempt}/
    checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt

    #replace "." with "_" in the bin list
    tr '\\.' '_' < $out_dir/${attempt}_DASTool_scaffolds2bin.txt > $out_dir/${attempt}_DASTool_scaffolds2bin_underscore.txt
    
done




#then import results to anvio for manual refining.



In [ ]:
# Collate all in FINAL bins
mkdir 05_binning/aa_FINAL_01_all
cp 05_binning/dastool/dastool_sample*/dastool_sample*_DASTool_bins/* 05_binning/aa_FINAL_01_all/
find ./05_binning/aa_FINAL_01_all -name "*-CLEAN*" -type f -size -500k -delete


# Remove contigs without SCGs
## Checking if contigs are correctly binned
Checking for single-copy marker genes --> CheckM on fasta with 1 less contig. If contigs are correctly binned, then completion should be less, contamination stay same.

In [ ]:
mkdir 05_binning/aa_FINAL_02_TRIMMED
mkdir 10_trimming
mkdir 07_checkm/zz_Trimming
conda activate binning


for bin in $(find ./05_binning/aa_FINAL_01_all/ -name "*.fa" -type f); do
    attempt=$(basename $bin)
    attempt=${attempt%.*}
    mkdir 10_trimming/$attempt
    mkdir 10_trimming/${attempt}_contigs

    # split multifasta into separate single fasta
    # https://gist.github.com/astatham/621901
    cat $bin | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fa")}
        print $0 > filename
    }'
    # move it to the contigs folder
    mv Utg*.fa 10_trimming/${attempt}_contigs/

    # sequentially cat all except one 
    for header in $(grep ">" $bin); do
        utg="${header:1}"
        cat $(find ./10_trimming/${attempt}_contigs/ -not -name "*$utg*" -type f) > 10_trimming/${attempt}/${attempt}_no_$utg.fa
    done
    cp $bin 10_trimming/${attempt}/${attempt}_no_deletions.fa

    checkm lineage_wf -x fa -t 64 --pplacer_threads 64 \
        10_trimming/$attempt/ \
        07_checkm/zz_Trimming/${attempt}
    checkm qa 07_checkm/zz_Trimming/${attempt}/lineage.ms 07_checkm/zz_Trimming/${attempt} --tab_table > 07_checkm/zz_Trimming/${attempt}/results.txt

    # dRep editing checkM results
    grep -v '\[' 07_checkm/zz_Trimming/${attempt}/results.txt > 07_checkm/zz_Trimming/${attempt}/results_edit.txt
    sed -i 's/Completeness/completeness/gI' 07_checkm/zz_Trimming/${attempt}/results_edit.txt
    sed -i 's/Contamination/contamination/gI' 07_checkm/zz_Trimming/${attempt}/results_edit.txt
    sed -i 's/\t/\,/gI' 07_checkm/zz_Trimming/${attempt}/results_edit.txt
    awk 'BEGIN{FS=OFS=","}{$1=$1".fa"}1' 07_checkm/zz_Trimming/${attempt}/results_edit.txt > 07_checkm/zz_Trimming/${attempt}/results_edit_genome.txt
    sed -i 's/Bin\ Id.fa/genome/gI' 07_checkm/zz_Trimming/${attempt}/results_edit_genome.txt
    mv 07_checkm/zz_Trimming/${attempt}/results_edit_genome.txt 07_checkm/zz_Trimming/${attempt}/results_edit_genome.csv

    Rscript get_core_contigs.R 07_checkm/zz_Trimming/${attempt}/results_edit_genome.csv

    # separate contigs with SCGs and contigs without
    input="07_checkm/zz_Trimming/${attempt}/core.txt"
    while IFS= read -r line
    do
        cat 10_trimming/${attempt}_contigs/$line >> 05_binning/aa_FINAL_02_TRIMMED/$attempt.fa
    done < "$input"


done









# dRep to dereplicate MAGs from bins of individual samples

Checkm, GTDBTK, dRep

In [ ]:
#CHECKM
attempt="aa_FINAL_02_TRIMMED"

conda deactivate
conda activate binning
checkm lineage_wf -x fa -t 32 --pplacer_threads 32 \
        05_binning/${attempt}/ \
        07_checkm/${attempt}/
checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt

#GTDBTK2
attempt="aa_FINAL_02_TRIMMED"
mkdir 06_taxa_gtdbtk2
conda deactivate
conda activate gtdbtk2

gtdbtk classify_wf \
    --genome_dir 05_binning/${attempt} \
    --out_dir 06_taxa_gtdbtk2/${attempt}/ \
    -x fa --cpus 32

# dRep editing checkM results
grep -v '\[' 07_checkm/${attempt}/results.txt > 07_checkm/${attempt}/results_edit.txt
sed -i 's/Completeness/completeness/gI' 07_checkm/${attempt}/results_edit.txt
sed -i 's/Contamination/contamination/gI' 07_checkm/${attempt}/results_edit.txt
sed -i 's/\t/\,/gI' 07_checkm/${attempt}/results_edit.txt
awk 'BEGIN{FS=OFS=","}{$1=$1".fa"}1' 07_checkm/${attempt}/results_edit.txt > 07_checkm/${attempt}/results_edit_genome.txt
sed -i 's/Bin\ Id.fa/genome/gI' 07_checkm/${attempt}/results_edit_genome.txt
mv 07_checkm/${attempt}/results_edit_genome.txt 07_checkm/${attempt}/results_edit_genome.csv

# dRep!
conda deactivate
conda activate dRep_dependencies
out_dir="./05_binning/aa_FINAL_03_TRIMMED_dRep_95"
mkdir $out_dir
rm -r $out_dir/*
dRep dereplicate $out_dir  -p 32 -g ./05_binning/${attempt}/*.fa --genomeInfo ./07_checkm/${attempt}/results_edit_genome.csv 

# add metabat_sample02.txt.007 because classified differently than all others in gtdbtk, has MQ assembly/binning, but not in dRep.
mkdir ./05_binning/aa_FINAL_04_TRIMMED_CONFIRM
cp ./05_binning/aa_FINAL_03_TRIMMED_dRep_95/dereplicated_genomes/* ./05_binning/aa_FINAL_04_TRIMMED_CONFIRM/
cp ./05_binning/aa_FINAL_02_TRIMMED/metabat_sample02.txt.007.fa ./05_binning/aa_FINAL_04_TRIMMED_CONFIRM/

In [ ]:

#CHECKM
attempt="aa_FINAL_04_TRIMMED_CONFIRM"

conda deactivate
conda activate binning
checkm lineage_wf -x fa -t 32 --pplacer_threads 32 \
        05_binning/${attempt}/ \
        07_checkm/${attempt}/
checkm qa 07_checkm/${attempt}/lineage.ms 07_checkm/${attempt} --tab_table > 07_checkm/${attempt}/results.txt

#GTDBTK2
attempt="aa_FINAL_04_TRIMMED_CONFIRM"
mkdir 06_taxa_gtdbtk2
conda deactivate
conda activate gtdbtk2

gtdbtk classify_wf \
    --genome_dir 05_binning/${attempt} \
    --out_dir 06_taxa_gtdbtk2/${attempt}/ \
    -x fa --cpus 32
